In [1]:
import os
from dotenv import load_dotenv
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, ImageClip, CompositeVideoClip, AudioFileClip
import pyttsx3 as tts
import whisper
from elevenlabs.client import ElevenLabs
from elevenlabs import generate, save
import google.generativeai as genai
load_dotenv()

client = ElevenLabs(api_key=os.getenv('ELEVENLABS_API_KEY'))
gemini_key = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=gemini_key)

In [ ]:
def generate_script(prompt, stories):
    model = genai.GenerativeModel('gemini-pro')
    default_prompt = """from now on return the output as regular text and the story should look
                    like it being narrated by someone, For an Example : There was once a boy fishing in the ocean
                    and he caught a fish, the fish was very big he was so happy, 
                    You have to write maximum 1000 characters I want a plain text no symbols just the script, 
                    also the script content must be in 50 seconds duration"""
    for i in range(stories):
        story = model.generate_content(f"{default_prompt},{prompt}")
        with open(f'story_{i}.txt', 'w') as f:
            f.write(story.text)

In [ ]:
prompt = "Scary Mystery Puzzle"
stories = 5
generate_script(prompt, stories)

In [ ]:
# def speak(text):
#     audio = generate(text,
#                      voice='Rachel',
#                      model = "eleven_multilingual_v2")
#     save(audio, 'samp.mp3')
    
def speak(text):
    engine = tts.init()
    engine.setProperty('rate', 150)
    engine.say(text)
    engine.runAndWait()
    engine.save_to_file(text, 'samp.mp3')

def get_files_in_directory(directory):
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]


with open('story_0.txt', 'r') as file:
    text = file.read()
    speak(text)

In [2]:
def speech_to_text(audio_file):
    model = whisper.load_model("base")
    data = model.transcribe(audio_file, word_timestamps=True)

    start = [data['segments'][i]['words'][j]['start'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]
    end = [data['segments'][i]['words'][j]['end'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]
    text = [data['segments'][i]['words'][j]['word'] for i in range(len(data['segments'])) for j in range(len(data['segments'][i]['words']))]

    return [start, end, text]

speech = speech_to_text("samp.mp3")

In [24]:
for l in range(len(speech[0])):
    print(((speech[0][l], speech[1][l]), speech[2][l]))

((0.0, 0.22), ' As')
((0.22, 0.3), ' the')
((0.3, 0.48), ' moon')
((0.48, 0.68), ' cast')
((0.68, 0.9), ' an')
((0.9, 1.08), ' eerie')
((1.08, 1.26), ' glow')
((1.26, 1.52), ' over')
((1.52, 1.66), ' the')
((1.66, 1.96), ' desolate')
((1.96, 2.28), ' streets,')
((2.78, 2.86), ' a')
((2.86, 3.0), ' lone')
((3.0, 3.26), ' figure')
((3.26, 3.56), ' emerges')
((3.56, 3.84), ' from')
((3.84, 4.0), ' the')
((4.0, 4.26), ' shadows,')
((4.78, 4.88), ' their')
((4.88, 5.12), ' footsteps')
((5.12, 5.68), ' echoing')
((5.68, 5.8), ' like')
((5.8, 5.94), ' a')
((5.94, 6.16), ' sinister')
((6.16, 6.56), ' heartbeat.')
((7.28, 7.52), ' In')
((7.52, 7.62), ' the')
((7.62, 7.76), ' dim')
((7.76, 7.94), ' light,')
((8.36, 8.48), ' we')
((8.48, 8.7), ' catch')
((8.7, 9.06), ' glimpses')
((9.06, 9.22), ' of')
((9.22, 9.34), ' their')
((9.34, 9.56), ' haunted')
((9.56, 10.02), ' expression,')
((10.48, 10.58), ' a')
((10.58, 10.86), ' reflection')
((10.86, 11.1), ' of')
((11.1, 11.2), ' the')
((11.2, 11.44

In [25]:
from moviepy.video.tools.subtitles import SubtitlesClip

def generate_video(): 
    with open('samp.txt', 'r') as file:
        text = file.read()
    
    lst = text.split(' ')

    clips = []

    generator = lambda txt: TextClip(txt, 
                                fontsize=70, 
                                color='white', 
                                bg_color='none', 
                                font='Arial-Bold',
                                method = 'caption',
                                size = (1920, 1080))

    clips = [CompositeVideoClip([ImageClip('Z_1.jpg').set_duration(0.5).set_position('center')], size=(1920, 1080)) for _ in lst]

    subs = [((speech[0][k], speech[1][k]), speech[2][k]) for k in range(len(speech[0]))]
        

    subtitles = SubtitlesClip(subs, generator)

    final = concatenate_videoclips(clips)
    audio = AudioFileClip("samp.mp3")
    final = final.set_audio(audio)
    final_complete = CompositeVideoClip([final, subtitles.set_position(('center', 'bottom'))])
    final_complete.write_videofile("output0.mp4", fps=24, codec = 'libx264')

In [26]:
generate_video()

Moviepy - Building video output0.mp4.
MoviePy - Writing audio in output0TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output0.mp4



Moviepy - Done !
Moviepy - video ready output0.mp4


In [7]:
import tortoise as t
from tortoise import api_fast

help(t)

Help on package tortoise:

NAME
    tortoise

PACKAGE CONTENTS
    api
    api_fast
    do_tts
    eval
    get_conditioning_latents
    is_this_from_tortoise
    models (package)
    read
    read_fast
    tts_stream
    utils (package)

FILE
    d:\projects\video-generator\myenv\lib\site-packages\tortoise_tts-3.0.0-py3.11.egg\tortoise\__init__.py




In [9]:
tts = api_fast.TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)
pcm_audio = tts.tts_with_preset("your text here", preset='fast')

Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.o

ModuleNotFoundError: No module named 'deepspeed'